#  prediction of address from the Invoice using LayoutLm model trained with Amazon and Flipkart Invoice details.

In [1]:
import pytesseract
from layoutlm_preprocess import *

In [2]:
from pdf2image import convert_from_path

In [3]:
pytesseract.pytesseract.tesseract_cmd = 'C:/Users/tintu.thomas02/AppData/Local/Tesseract-OCR/tesseract.exe'

In [4]:
pages = convert_from_path('invoice.pdf',poppler_path=r"C:\Users\tintu.thomas02\Downloads\Release-22.04.0-0 (1)\poppler-22.04.0\Library\bin")
for image in pages:
    image.save('output.png')

In [5]:
from torch.nn import CrossEntropyLoss
def get_labels(path):
    with open(path, "r") as f:
        labels = f.read().splitlines()
    if "O" not in labels:
        labels = ["O"] + labels
    return labels

labels = get_labels("Invoiceannotation/labels.txt")
num_labels = len(labels)
label_map = {i: label for i, label in enumerate(labels)}
# Use cross entropy ignore index as padding label id so that only real label ids contribute to the loss later
pad_token_label_id = CrossEntropyLoss().ignore_index

In [6]:
PATH='./layoutlmnew15.pt'
models = LayoutLMForTokenClassification.from_pretrained("microsoft/layoutlm-base-uncased", num_labels=num_labels)
models.load_state_dict(torch.load(PATH))

Some weights of the model checkpoint at microsoft/layoutlm-base-uncased were not used when initializing LayoutLMForTokenClassification: ['cls.predictions.decoder.weight', 'cls.predictions.bias', 'cls.predictions.decoder.bias', 'cls.predictions.transform.LayerNorm.bias', 'cls.predictions.transform.dense.bias', 'cls.predictions.transform.LayerNorm.weight', 'cls.predictions.transform.dense.weight']
- This IS expected if you are initializing LayoutLMForTokenClassification from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing LayoutLMForTokenClassification from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).
Some weights of LayoutLMForTokenClassification were not initialized from the model checkpoint at microsoft

<All keys matched successfully>

In [7]:
image, words, boxes, actual_boxes = preprocess("output.png")
word_level_predictions, final_boxes=convert_to_features(image, words, boxes, actual_boxes, models)
draw = ImageDraw.Draw(image)
font = ImageFont.load_default()
def iob_to_label(label):
    if label != 'O':
        return label[2:]
    else:
        return ""
label2color = {'QUESTION':'blue','question':'blue','ANSWER':'green','answer':'green','BILLING_ADDRESS_HEADER':'orange','billing_address_header':'orange','BILLINGADDRESSDETAILS':'violet','billingaddressdetails':'violet','HEADER':'red','SOLDBYHEADER':'black','soldbyheader':'black','SOLDBYDETAILS':'green','soldbydetails':'green','SHIPPINGADDRESSHEADER':'yellow','shippingaddressheader':'yellow','SHIPPINGADDRESSDETAILS':'magenta','shippingaddressdetails':'magenta','OTHERS':'black','':'violet','header':'red','others':'black'}
# label2color = {'question':'blue', 'answer':'green', 'header':'orange', '':'violet'}
for prediction, box in zip(word_level_predictions, final_boxes):
    predicted_label = iob_to_label(label_map[prediction]).lower()
    draw.rectangle(box, outline=label2color[predicted_label])
    draw.text((box[0] + 10, box[1] - 10), text=predicted_label, fill=label2color[predicted_label], font=font)

[31, 34, 34, 34, 34, 34, 10, 20, 20, 29, 29, 29, 19, 29, 29, 29, 29, 19, 29, 29, 29, 29, 11, 21, 29, 29, 11, 29, 29, 29, 29, 29, 29, 29, 29, 29, 29, 29, 29, 29, 29, 29, 29, 29, 21, 29, 29, 29, 29, 21, 29, 29, 29, 29, 29, 29, 21, 29, 29, 29, 29, 29, 29, 29, 29, 29, 21, 29, 29, 21, 29, 29, 29, 36, 36, 29, 19, 29, 29, 31, 29, 31, 31, 31, 6, 36, 16, 31, 33, 31, 31, 31, 31, 31, 31, 31, 31, 31, 31, 31, 6, 6, 26, 16, 16, 31, 31, 31, 31, 31, 36, 31, 16, 31, 31, 31, 31, 31, 31, 31, 31, 31, 31, 36, 31, 16, 31, 31, 31, 31, 31, 31, 34, 34, 34, 34, 34, 34, 34, 34, 34, 34, 34, 34, 34, 34, 34, 34, 38, 38, 38, 32, 32, 32, 32, 32, 32, 32, 32, 32, 32, 32, 32, 21, 32, 32, 32, 32, 21, 32, 32, 32, 32, 32, 32, 32, 32, 32, 32, 32, 32, 21, 32, 21, 32, 32, 32, 32, 32, 32, 32, 32, 37, 32, 38, 38, 38, 37, 37, 37, 37, 37, 37, 37, 37, 37, 37, 37, 37, 37, 37, 37, 37, 21, 37, 37, 37, 37, 37, 21, 37, 37, 37, 37, 37, 37, 37, 37, 21, 37, 21, 37, 37, 37, 37, 37, 37, 37, 37, 37, 37, 37, 37, 37, 37, 37, 37, 37, 37, 37, 37

In [8]:
import cv2
def bounding_box_img(img,bbox):
            x_min, y_min, x_max, y_max = bbox
            bbox_obj = img[y_min-4:y_max+4, x_min-4:x_max+4]
            return bbox_obj
flag = 0
# iob_to_label(label_map[word_level_predictions[i]]).lower() == "billing_address_header" or 
img = cv2.imread("output.png")
for i in range(len(word_level_predictions)):
    if iob_to_label(label_map[word_level_predictions[i]]).lower() == "billingaddressdetails":
        cropped_img = bounding_box_img(img,final_boxes[i])
        text = pytesseract.image_to_string(cropped_img)
        print(text)

Tintu

Thomas

Malabar


Hostel,

Hostel,

Kalavath

Road,

Palarivattom,

Opposite

To

Powerskill

Engineering

Solutions

Pvt

Limited

Ernakulam,

KERALA,






Code:32

Code:32

